# BIG DATA ANALYSIS 

## Telechargement des bibliothèques utiles

In [1]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
from pyspark.sql import SparkSession

def get_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    return data

tickers = ["AAPL", "GOOGL", "AMZN", "MSFT"]
start_date = "2020-01-01"
end_date = "2021-01-01"

# Télécharger les données
stock_data = get_stock_data(tickers, start_date, end_date)

# Vérification des données pandas
stock_data.info()


[*********************100%***********************]  4 of 4 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2020-01-02 to 2020-12-31
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (Adj Close, AAPL)   253 non-null    float64
 1   (Adj Close, AMZN)   253 non-null    float64
 2   (Adj Close, GOOGL)  253 non-null    float64
 3   (Adj Close, MSFT)   253 non-null    float64
 4   (Close, AAPL)       253 non-null    float64
 5   (Close, AMZN)       253 non-null    float64
 6   (Close, GOOGL)      253 non-null    float64
 7   (Close, MSFT)       253 non-null    float64
 8   (High, AAPL)        253 non-null    float64
 9   (High, AMZN)        253 non-null    float64
 10  (High, GOOGL)       253 non-null    float64
 11  (High, MSFT)        253 non-null    float64
 12  (Low, AAPL)         253 non-null    float64
 13  (Low, AMZN)         253 non-null    float64
 14  (Low, GOOGL)        253 non-null    float64
 15  (Low, MSFT)         253 non-null    fl

In [13]:
import yfinance as yf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, DateType

# Initialiser SparkSession
spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()

# Fonction pour télécharger les données d'un ticker
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.reset_index(inplace=True)  # Ajouter la colonne Date
    return data

# Fonction pour convertir un DataFrame pandas en DataFrame Spark
def convert_to_spark_df(pandas_df, ticker):
    schema = StructType([
        StructField("Date", DateType(), True),  # Colonne Date
        StructField(f"Adj_Close_{ticker}", FloatType(), True),
        StructField(f"Close_{ticker}", FloatType(), True),
        StructField(f"High_{ticker}", FloatType(), True),
        StructField(f"Low_{ticker}", FloatType(), True),
        StructField(f"Open_{ticker}", FloatType(), True),
        StructField(f"Volume_{ticker}", IntegerType(), True),
    ])
    
    # Renommer les colonnes pandas pour correspondre au schéma
    pandas_df = pandas_df.rename(columns={
        "Adj Close": f"Adj_Close_{ticker}",
        "Close": f"Close_{ticker}",
        "High": f"High_{ticker}",
        "Low": f"Low_{ticker}",
        "Open": f"Open_{ticker}",
        "Volume": f"Volume_{ticker}",
    })
    
    # Créer le DataFrame Spark
    spark_df = spark.createDataFrame(pandas_df, schema=schema)
    return spark_df

# Liste des tickers et période
tickers = ["AAPL", "GOOGL", "AMZN", "MSFT"]
start_date = "2020-01-01"
end_date = "2021-01-01"

# Création d'un DataFrame Spark par ticker
spark_dfs = {}  # Dictionnaire pour stocker les DataFrames Spark
for ticker in tickers:
    print(f"Téléchargement des données pour {ticker}...")
    pandas_df = get_stock_data(ticker, start_date, end_date)  # Télécharger les données
    spark_df = convert_to_spark_df(pandas_df, ticker)  # Convertir en DataFrame Spark
    spark_dfs[ticker] = spark_df  # Stocker dans le dictionnaire

# Vérification d'un DataFrame Spark (exemple : AAPL)
print("Schéma du DataFrame Spark pour AAPL :")
spark_dfs["AAPL"].printSchema()
spark_dfs["AAPL"]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Téléchargement des données pour AAPL...
Téléchargement des données pour GOOGL...
Téléchargement des données pour AMZN...



[*********************100%***********************]  1 of 1 completed

Téléchargement des données pour MSFT...
Schéma du DataFrame Spark pour AAPL :
root
 |-- Date: date (nullable = true)
 |-- Adj_Close_AAPL: float (nullable = true)
 |-- Close_AAPL: float (nullable = true)
 |-- High_AAPL: float (nullable = true)
 |-- Low_AAPL: float (nullable = true)
 |-- Open_AAPL: float (nullable = true)
 |-- Volume_AAPL: integer (nullable = true)



DataFrame[Date: date, Adj_Close_AAPL: float, Close_AAPL: float, High_AAPL: float, Low_AAPL: float, Open_AAPL: float, Volume_AAPL: int]